In [1]:
# Imports
import pandas as pd
import numpy as np
from pyproj import Transformer

## Read hdb carpark location xcoord and ycoord obtained via https://beta.data.gov.sg/collections/148/datasets/d_23f946fa557947f93a8043bbef41dd09/view

The x and y coordinates provided are in svy21 coordinate system. To facilitate location mapping with WGS84 coordinates, we need to use a library to perform the covnersion
As a result there is a need to translate the given coordinate from svy21 to WGS84 lat lon equivalent

In [2]:
hdb_carpark_location = pd.read_csv("../data/HDBCarparkInformation.csv")
hdb_carpark_location.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.0,N
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N
4,AK31,BLK 302/348 ANG MO KIO STREET 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N


In [3]:
hdb_carpark_location.isna().sum()

car_park_no               0
address                   0
x_coord                   0
y_coord                   0
car_park_type             0
type_of_parking_system    0
short_term_parking        0
free_parking              0
night_parking             0
car_park_decks            0
gantry_height             0
car_park_basement         0
dtype: int64

## Test conversion from svy21 to wgs84

Using second datapoint as reference, we expect the conversion of x,y coordinates to lat lon would be close to 1.321227, 103.884940.

In [4]:
# Test conversion from SVY21 coordinate system to WGS84
svy21_to_wgs84 = Transformer.from_crs("EPSG:3414", "EPSG:4326")
x,y = 33758.4143,33695.5198
print(svy21_to_wgs84.transform(x,y))

(1.3215730972753876, 103.88449581937945)


## Convert existing SVY21 (EPSG:3414) coordinate system to WGS84 (EPSG:4326)

In [5]:
def convert_SVY21_to_WGS84(row):
    svy21_to_wgs84 = Transformer.from_crs("EPSG:3414", "EPSG:4326")
    x2,y2 = svy21_to_wgs84.transform(row['x_coord'], row['y_coord'])
    return pd.Series([x2,y2])

hdb_carpark_location[['Lat','Lon']] = hdb_carpark_location.apply(convert_SVY21_to_WGS84, axis=1)

In [6]:
hdb_carpark_location.drop(columns=["x_coord","y_coord"], inplace=True)
hdb_carpark_location.set_index("car_park_no", inplace=True)
hdb_carpark_location.head()

,address,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement,Lat,Lon
car_park_no,,,,,,,,,,,
ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y,1.290431,103.864682
ACM,BLK 98A ALJUNIED CRESCENT,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N,1.321573,103.884496
AH1,BLK 101 JALAN DUSUN,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.0,N,1.280870,103.891727
AK19,BLOCK 253 ANG MO KIO STREET 21,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N,1.271172,103.932271
AK31,BLK 302/348 ANG MO KIO STREET 31,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N,1.282898,103.929320


In [7]:
# Save converted csv file for reference
hdb_carpark_location.to_csv("../data/HDBCarparkInformationWGS84.csv", index=True)